# TIFF Combiner Notebook
This notebook combines multiple TIFF files from a parent folder or its subfolders into single multi-page TIFF files.
- If the parent folder contains TIFF files directly, all are combined into one TIFF named after the parent folder.
- If the parent folder has subfolders (each with TIFF files), each subfolder's TIFFs are combined into one multi-page TIFF named after the subfolder.
- Outputs are saved to a user-defined output directory.

## Configuration Variables
- `tif_folder_path` : Path to the parent folder containing TIFF files or subfolders with TIFFs.
- `output_folder_path` : Path where combined TIFF files will be saved.

In [1]:
# Import necessary libraries
from PIL import Image
import os
from pathlib import Path
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Configuration variables
tif_folder_path = r"C:/Users/jeeb/Downloads/masked"  # TODO: Change this to your parent folder path
output_folder_path = r"C:/Users/jeeb/Downloads/masked/aggregated"  # TODO: Change this to your output folder

output_dir = Path(output_folder_path)
output_dir.mkdir(parents=True, exist_ok=True)

def combine_tiffs(tiffs, output_path):
    """
    Combine a list of TIFF file paths into a single multi-page TIFF.
    :param tiffs: List of TIFF file paths (strings or Path objects)
    :param output_path: Output TIFF file path
    """
    if not tiffs:
        logging.warning(f"No TIFF files to combine for {output_path}")
        return
    try:
        images = [Image.open(str(t)) for t in sorted(tiffs)]  # Sorting ensures page order
        first_img, *rest_imgs = images
        first_img.save(output_path, save_all=True, append_images=rest_imgs)
        logging.info(f"Saved combined TIFF to {output_path}")
    except Exception as e:
        logging.error(f"Failed to combine TIFFs to {output_path}: {e}")

def main():
    parent_dir = Path(tif_folder_path)
    if not parent_dir.exists() or not parent_dir.is_dir():
        logging.error(f"Provided TIFF folder path '{parent_dir}' does not exist or is not a directory.")
        return

    # Find TIFF files in parent folder (non-recursive)
    parent_tiffs = list(parent_dir.glob('*.tif')) + list(parent_dir.glob('*.tiff'))

    if parent_tiffs:
        # Combine all TIFFs directly under parent folder
        output_file = output_dir / (parent_dir.name + '.tif')
        logging.info(f"Combining {len(parent_tiffs)} TIFF files directly under parent folder...")
        combine_tiffs(parent_tiffs, output_file)
    else:
        # No TIFFs at parent level, so traverse subfolders
        subfolders = [d for d in parent_dir.iterdir() if d.is_dir()]
        if not subfolders:
            logging.warning(f"No TIFF files found in parent folder or subfolders.")
            return
        for sub in sorted(subfolders):
            sub_tiffs = list(sub.glob('*.tif')) + list(sub.glob('*.tiff'))
            if sub_tiffs:
                output_file = output_dir / (sub.name + '.tif')
                logging.info(f"Combining {len(sub_tiffs)} TIFF files in subfolder: {sub.name}")
                combine_tiffs(sub_tiffs, output_file)
            else:
                logging.warning(f"No TIFF files found in subfolder: {sub.name}")

# Run main logic
if __name__ == '__main__':
    main()

INFO: Combining 39 TIFF files in subfolder: masked (1)_pages3
INFO: Saved combined TIFF to C:\Users\jeeb\Downloads\masked\aggregated\masked (1)_pages3.tif
INFO: Combining 53 TIFF files in subfolder: masked (2)_pages6
INFO: Saved combined TIFF to C:\Users\jeeb\Downloads\masked\aggregated\masked (2)_pages6.tif
INFO: Combining 40 TIFF files in subfolder: masked (3)_pages4
INFO: Saved combined TIFF to C:\Users\jeeb\Downloads\masked\aggregated\masked (3)_pages4.tif
INFO: Combining 21 TIFF files in subfolder: masked (5)_pages2
INFO: Saved combined TIFF to C:\Users\jeeb\Downloads\masked\aggregated\masked (5)_pages2.tif
INFO: Combining 16 TIFF files in subfolder: masked(4)_pages2
INFO: Saved combined TIFF to C:\Users\jeeb\Downloads\masked\aggregated\masked(4)_pages2.tif


### Instructions
- Set the `tif_folder_path` and `output_folder_path` to your actual folder paths.
- Run the notebook to combine TIFF files as per the logic.
- Logs will show progress, warnings, and errors.
- Combined TIFF files will be saved in the output folder.